# Neptune + Catalyst

## Introduction

This guide will show you how to:

* Create a `NeptuneLogger`,
* Log a catalyst model using `NeptuneLogger`.

## Before you start

This notebook example lets you try out Neptune as an anonymous user, with zero setup.

* If you are running the notebook on your local machine, you need to have [Python](https://www.python.org/downloads/) and [pip](https://pypi.org/project/pip/) installed.
* If you want to see the example recorded to your own workspace instead:
    * Create a Neptune account → [Take me to registration](https://neptune.ai/register)
    * Create a Neptune project that you will use for tracking metadata → [Tell me more about projects](https://docs.neptune.ai/administration/projects)

## Install Neptune and dependencies

In [ ]:
! pip install -U catalyst ipywidgets neptune-client tensorboard torchvision

## Import libraries

In [ ]:
import os
from collections import OrderedDict

import neptune.new as neptune

from catalyst import dl
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from torch import nn, optim
from torch.utils.data import DataLoader

## Prepare hyper-parameters

In [ ]:
my_hparams = {"lr": 0.07, "batch_size": 32}

They will be logged automatically.

## Prepare model, criterion, optimizer and data loaders

Define components needed for the Catalyst Runner.

In [ ]:
model = nn.Sequential(nn.Flatten(), nn.Linear(28 * 28, 10))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), my_hparams["lr"])
loaders = OrderedDict(
    {
        "training": DataLoader(
            MNIST(os.getcwd(), train=True, download=True, transform=ToTensor()),
            batch_size=my_hparams["batch_size"],
        ),
        "validation": DataLoader(
            MNIST(os.getcwd(), train=False, download=True, transform=ToTensor()),
            batch_size=my_hparams["batch_size"],
        ),
    }
)

Notice two keys: `"training"` and `"validation"`. These names are used in Neptune as namespaces to organize your run.

## Create runner

In [ ]:
my_runner = dl.SupervisedRunner()

## Create NeptuneLogger

To connect your script to Neptune and create a new run, we tell Neptune:
* **Who you are** - with a Neptune API token
* **Where to send your data** - to a Neptune project

The cell below lets you record data to the public project [common/catalyst-integration](https://app.neptune.ai/common/catalyst-integration) as an anonymous user.

In [ ]:
from catalyst import dl

neptune_logger = dl.NeptuneLogger(
    api_token=neptune.ANONYMOUS_API_TOKEN,
    project="common/catalyst-integration",
    tags=["notebook-example", "quickstart"],
)

Link above is a link to the run. Click on it, and leave the run tab open - you will get back to it when you start model training.

Few explanations:
1. You need to pass a project to the `NeptuneLogger()`, to inform it where to log metadata. Project is a string of this form: `my_workspace/my_project`.
1. There are more parameters to customize logger behavior, check [Catalyst docs](https://catalyst-team.github.io/catalyst/api/loggers.html#neptunelogger) for more details.

----

**Note**

Instead of logging data to the public project `"common/catalyst-integration"` as an anonymous user `"neptuner"` you can log it to your own project.

To do that, replace the code above with the following:

```python
from getpass import getpass

neptune_logger = dl.NeptuneLogger(
    api_token=getpass("Enter your Neptune API token: "),
    project="workspace-name/project-name",  # replace with your own
)
```

For example, if your workspace name is `ml-team` and the project name is `classification`, the project argument is: `project="ml-team/classification"`.

To find your API token and project name, [log in to Neptune](https://app.neptune.ai/).
- In the top-right corner, click your avatar and select **Get your API token**.
- To find and copy your project name, navigate to the project, then click **Settings** → **Properties**.

## Pass neptune_logger to the train method of the Runner

In [ ]:
my_runner.train(
    loggers={"neptune": neptune_logger},
    hparams=my_hparams,
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    loaders=loaders,
    num_epochs=5,
    callbacks=[
        dl.AccuracyCallback(input_key="logits", target_key="targets", topk=[1]),
        dl.CheckpointCallback(
            logdir="checkpoints",
            loader_key="validation",
            metric_key="loss",
            minimize=True,
        ),
    ],
    valid_loader="validation",
    valid_metric="loss",
    minimize_valid_metric=True,
)

Few explanations:

1. `train` method executes model training, so here you start logging metadata to Neptune.
1. `neptune_logger` is passed to the `loggers` parameter.
1. Notice that `my_hparams` are also passed, to that they can be logged as well.

## Log best model

After training, use `log_artifact()` method to log model checkpoint.

In [ ]:
my_runner.log_artifact(
    path_to_artifact="./checkpoints/model.best.pth", tag="best_model", scope="experiment"
)

## Stop logging

Once you are done logging, stop tracking the run.

In [ ]:
neptune_logger.run.stop()

## Analyze logged metadata in the Neptune app

Go to the run link and explore metadata (metrics, hparams, model checkpoint) that were logged to the run in Neptune.

Link should look like this: https://app.neptune.ai/common/catalyst-integration/e/CATALYST-1486